In [1]:
import nbimporter
from my_test import load_and_display_heads, reduce_tracking_data_direct
game_data, play_data, player_play_data, player_data, tracking_data = load_and_display_heads()
reduced_tracking_data = reduce_tracking_data_direct(tracking_data, 30)

In [2]:
play_data = play_data.merge(reduced_tracking_data, on='playId', how='left')
# print(play_data)

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

def generate_x_data(data):
    
    x_data = []
    # Step 2: Normalize numerical features (e.g., x, y, speed, acceleration)
    numerical_columns = ['x', 'y', 's', 'a', 'yardsToGo', 'down', 'preSnapHomeScore', 'preSnapVisitorScore', 'quarter']
    scaler = StandardScaler()
    data[numerical_columns] = scaler.fit_transform(data[numerical_columns])

    # Step 3: One-hot encode categorical features (playDirection, event, position)
    # data = pd.get_dummies(data, columns=['playDirection', 'event', 'position'], drop_first=False)
    
    # Step 4: Handle missing values (e.g., filling with zeros)
    data = data.fillna(0)
    
    # Iterate through plays
    for play_id, play_df in data.groupby("playId"):
        for _, play_info in play_df.iterrows():
            
            # Extract situation features
            situation_features = {
                "down": play_info["down"],
                "distance": play_info["yardsToGo"],
                "score_differential": play_info["preSnapHomeScore"] - play_info["preSnapVisitorScore"],
                "quarter": play_info["quarter"],
                # "game_clock": play_info["gameClock"]
            }

            # Get player tracking data at the snap
            # snap_frame = play_df[play_df["frameType"] == "SNAP"]
            # if snap_frame.empty:
            #     continue
            # for _, row in snap_frame.iterrows():
            player_features = {
                "x": play_info["x"],
                "y": play_info["y"],
                "s": play_info["s"],
                "dir": play_info["dir"],
                "o": play_info["o"],
                "team": play_info["club"],
                # "player_role": play_info["position"]
            }
            # Create X (features) for the play
            x_data.append({
                "situation": situation_features,
                "players": player_features
            })
    return x_data

In [ ]:
data = play_data
x_data = generate_x_data(data)
# print(x_data)